In [ ]:
import tensorflow as tf
import numpy as np
import datetime
# from sklearn.model_selection import StratifiedGroupKFold, KFold

from utils import get_id_label, to_npy_array
from model import DecouplingConvNet

train_txt = '/data1/jianghai/DECT/txt/binary/train.txt'
test_txt = '/data1/jianghai/DECT/txt/binary/test.txt'
train_npy_path_list, train_label_list = get_id_label(train_txt, flag='train')
test_npy_path_list, test_label_list = get_id_label(test_txt, flag='test')

train_dataset = tf.data.Dataset.from_tensor_slices((to_npy_array(train_npy_path_list), train_label_list))
test_dataset = tf.data.Dataset.from_tensor_slices((to_npy_array(test_npy_path_list), test_label_list))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

model = DecouplingConvNet()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Accuracy(name='train_accuracy')
train_auc = tf.keras.metrics.AUC(name='train_auc')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.Accuracy(name='test_accuracy')
test_auc = tf.keras.metrics.AUC(name='test_auc')

In [ ]:
@tf.function
def train_step(array, labels):
    with tf.GradientTape() as tape:
        predictions = model(array, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    train_auc(labels, predictions)
@tf.function
def test_step(array, labels):
    predictions = model(array, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    test_auc(labels, predictions)

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = '/data1/jianghai/DECT/code/keras/tensorboard/' + current_time + '/train'
test_log_dir = '/data1/jianghai/DECT/code/keras/tensorboard/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1, 
    patience=50, 
    mode='max', 
    restore_best_weights=True
)

In [ ]:
# from tensorboard import S
EPOCHS = 3000
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    train_auc.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    test_auc.reset_states()

    for train_array, train_labels in train_dataset:
        train_step(train_array, train_labels)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
        tf.summary.scalar('auc', train_auc.result(), step=epoch)

    for test_array, test_labels in test_dataset:
        test_step(test_array, test_labels)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
        tf.summary.scalar('auc', test_auc.result(), step=epoch)
    
    print(
        f'Epoch {epoch + 1}, '
        # f'Train Loss: {train_loss.result()}, '
        # f'Train Accuracy: {train_accuracy.result()}, '
        f'Train AUC: {train_auc.result()}, '
        # f'Test Loss: {test_loss.result()}, '
        # f'Test Accuracy: {test_accuracy.result()}, '
        f'Test AUC: {test_auc.result()}'
    )

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'), 
    tf.keras.metrics.AUC(name='auc'), 
    tf.keras.metrics.AUC(name='prc', curve='PR')
]
model = DecouplingConvNet()
loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss=loss_object, metrics=METRICS)
# model.build(input_shape=(1, 110, 110, 11))
# model.summary()
model.fit(train_dataset, epochs=300)